In [3]:
import openpyxl, re
from openpyxl import Workbook

#### We need the name of the workbook we are to modify.

In [7]:
file_name = input('Enter in the filename, in single or double quotes, of the workbook you\'d like to modify: ')
book = openpyxl.load_workbook(file_name)

Enter in the filename, in single or double quotes, of the workbook you'd like to modify: "cereal.xlsx"


#### First, we will load the first spreadsheet in the workbook.

In [8]:
sheet = book.active

#### We create a new workbook, in which we place the restructured data.

In [9]:
structured_book = Workbook()
structured_sheet = structured_book.active
structured_sheet.title = 'Restructured Nutrition Data'

#### We create a method for checking possible product description matches, to be used in the main loop.

In [10]:
def getProductDescription(tokens):
    ret = ''
    for token in tokens:
        if any(char.isdigit() for char in token) or len(token) == 0 or 'except as noted' in token:
            continue
        else:
            if ret == '':
                ret = token.strip()
            else:
                ret += ', ' + token.strip()
    return ret.strip().strip(':')

#### We create a method for extracting serving size information, to be used in the main loop.

In [22]:
def getServingSize(tokens):
    for token in tokens: # loop through each token
        if any(char.isdigit() for char in token) and '(' not in token: # looking for the token with the serving size data...
            return token.replace('.', '').strip().strip(':')

#### Here we have the main loop. For each row which contains nutrition information, we in turn must find the associated (1) Product Category, (2) Serving Size, and try to find its (3) Product Description, (4) Type, and (5) Brand Name, if they exist for the row's entry.

In [23]:
last_product_category = 0
last_product_description = ''
last_type = 0
last_brand_name = 0
last_serving_size = 0

# pointers to rows which contain these types of data points
product_category_rows = []
product_description_rows = []
brand_name_rows = []

# a pointer to where we are in the newly formatted spreadsheet
structured_sheet_pointer = 0

# the main loop
for i, row in enumerate(sheet): # check each row for nutrition data
    if row[1].value: # nutrition data always starts in row 2. if there is data in row 2, there is data in rows 2-8
        
        # get all the numerical nutrition data into the new sheet
        nutrition_data = [sheet.cell(row=i+1, column=j).value for j in range(2,9)]
        for j in range(6,13):
            structured_sheet.cell(row=structured_sheet_pointer+1, column=j).value = nutrition_data[j-6]
        
        # find and store the product category (each entry must have one)
        for j in range(i, last_product_category, -1): # check all previous rows up to last known product category row
            # grab data in current cell, in the first column
            current_cell = sheet.cell(row=j, column=1)
            # if the cell is bold, we know that it contains product category information
            if current_cell.style.font.bold and '(cont.)' not in current_cell.value:
                # set the first column in the current row of the restructured sheet to this product category
                # TODO: tidy this expression up so that it grabs the product category only
                structured_sheet.cell(row=structured_sheet_pointer+1, column=1).value = current_cell.value.split(',')[0]
                last_product_category = j-1
                product_category_rows.append(j)
                break
                
        # find and store the product description (if it exists)
        for j in range(i, 0, -1): # check all previous rows up to last product category row
            # grab data in current cell, in the first column
            current_cell = sheet.cell(row=j, column=1)
            # if the cell contains some text preceding a semicolon, it must contain a product description
            if current_cell.value and ':' not in current_cell.value:
                continue
            if current_cell.style.font.bold:
                product_description = 'NA'
                break
            else: # there must be a product description
                if current_cell.value: # if the cell contains text
                    cell_text = re.sub('\(.*\)', '', current_cell.value) # remove all parenthesized text
                    cell_tokens = cell_text.split(',') # split the cell's text into chunks, separated by comma occurence
                    product_description = getProductDescription(cell_tokens) # outsource implementation to method
                    structured_sheet.cell(row=structured_sheet_pointer+1, column=2).value = product_description
                    if product_description == '':
                        continue
                    last_product_description = product_description
                    product_description_rows.append(j)
                    break
                
        if not structured_sheet.cell(row=structured_sheet_pointer+1, column=2).value:
            structured_sheet.cell(row=structured_sheet_pointer+1, column=2).value = 'NA'
        
        # find and store the type (if it exists)
        for j in range(i, 0, -1): # check all previous rows up to the last product category row
            # grab dat ain current cell, in the first column
            current_cell = sheet.cell(row=j, column=1)

        # find and store the brand name (if it exists)
        for j in range(i+1, 0, -1):
            # grab data in current cell, in the first column            
            current_cell = sheet.cell(row=j, column=1)
            # if the cell contains parenthesized text (not equal to cont.), it must contain brand name information
            if current_cell.value and current_cell.value.find('(') != -1 and '(cont.)' not in current_cell.value:
                # get the cell's text within parentheses; this is the brand name
                brand_name = current_cell.value[current_cell.value.find("(")+1:current_cell.value.find(")")]
                structured_sheet.cell(row=structured_sheet_pointer+1, column=4).value = brand_name
                brand_name_rows.append(j)
                break
                
            elif j in product_category_rows or j in product_description_rows:
                # there cannot be any brand name information before here
                structured_sheet.cell(row=structured_sheet_pointer+1, column=4).value = 'NA'
                break
            
        # find and store the serving size (each entry must have one)
        for j in range(i+1, 0, -1): # check all previous rows up to the last product category row
            # grab data in current cell, in the first column            
            current_cell = sheet.cell(row=j, column=1)
            # if the cell contains some numerical data, it will pertain to the serving size
            if current_cell.value and any(char.isdigit() for char in current_cell.value) and j == i+1:
                # split the cell's text by comma, looking for the token with numerical data
                serving_size = getServingSize(current_cell.value.split(','))
                
                if serving_size:
                    structured_sheet.cell(row=structured_sheet_pointer+1, column=5).value = serving_size
                    break
                else:
                    continue
                    
            elif j in product_description_rows or j in product_category_rows:
                # split the cell's text by comma, looking for the token with numerical data
                serving_size = getServingSize(current_cell.value.split(','))
                
                if serving_size:
                    structured_sheet.cell(row=structured_sheet_pointer+1, column=5).value = serving_size
                    break
                else:
                    continue
        
        structured_sheet_pointer += 1

structured_book.save('restructured_' + file_name)

In [19]:
column_widths = []
for j, row in enumerate(structured_sheet):
    for i, cell in enumerate(row):
        if len(column_widths) > i:
            try:
                if len(cell.value) > column_widths[i]:
                    column_widths[i] = len(cell.value)
            except TypeError:
                pass
        else:
            column_widths += [len(cell.value)]

structured_sheet.column_dimensions["A"].width = column_widths[0] + 2.0
structured_sheet.column_dimensions["B"].width = column_widths[1] + 2.0
structured_sheet.column_dimensions["C"].width = column_widths[2] + 2.0
structured_sheet.column_dimensions["D"].width = column_widths[3] + 2.0
structured_sheet.column_dimensions["E"].width = column_widths[4] + 2.0
structured_sheet.column_dimensions["F"].width = 6.0
structured_sheet.column_dimensions["G"].width = 6.0
structured_sheet.column_dimensions["H"].width = 6.0
structured_sheet.column_dimensions["I"].width = 6.0
structured_sheet.column_dimensions["J"].width = 6.0
structured_sheet.column_dimensions["K"].width = 6.0
structured_sheet.column_dimensions["L"].width = 6.0


structured_book.save('restructured_' + file_name)